# Imports and Installs

In [1]:
# install latest version of polyglot language detection library for stretch
# !pip install -U git+https://github.com/aboSamoor/polyglot.git@master

# !pip install plotly

In [1]:
import pandas as pd
import numpy as np 
#import pandas_profiling 
from sklearn import preprocessing # for category encoder
# from polyglot.detect import Detector # for language detection stretch
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
# much more efficient for larger files like Nearest Neighbors which the model
import joblib
import plotly.express as px
import plotly.graph_objects as go

# EDA & PROFILE REPORT

In [2]:
# Read in data
raw_df = pd.read_csv('https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv')
raw_df = raw_df.dropna() # drop null values

In [3]:
raw_df.shape

(232725, 18)

In [4]:
raw_df.head(1)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.91,0.0,C#,0.346,-1.828,Major,0.0525,166.969,4/4,0.814


In [5]:
raw_df = raw_df.drop_duplicates(subset='track_name')

In [6]:
raw_df.shape

(148615, 18)

In [9]:
raw_df.to_csv('spotify_unique_track_id.csv', index=False)

In [4]:
# def create_web_chart(df):
#     music_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']
    
#     fig = go.Figure()

#     for idx, row in df.iterrows():
#         # vals = list(df.iloc[idx][music_features].values)
#         row_df = pd.DataFrame(df.loc[idx][music_features].T).reset_index().rename(columns={idx:'values', 'index':'feature'})
#         fig.add_trace(go.Scatterpolar(
#                                     r=row_df['values'],
#                                     theta=row_df['feature'],
#                                     fill='toself',
#                                     name= f'{row.artist_name} - {row.track_name}'
#                     ))
#     # fig.show()
#     return fig

# fig = create_web_chart(df.iloc[0:5])
# fig.show()

In [5]:
# first_song_values = list(df.iloc[0][music_features].values)
# second_song_values = list(df.iloc[1][music_features].values)

# fig = px.bar(first_song_df, x='feature', y="values", color='feature')
# fig.show()

In [6]:
# fig.add_trace(go.Scatterpolar(
#       r=second_song_df['values'],
#       theta=second_song_df['feature'],
#       fill='toself',
#       name='Second Song'
# ))

# fig.update_layout(
#   polar=dict(
#     radialaxis=dict(
#       visible=True,
#       range=[0, 1]
#     )),
#   showlegend=True
# )

# fig.show()

# DATA ENGINEERING CATEGORY ENCODING OF OBJECT FEATURES
note: might need to create another column of encoded languages
resources below: https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/


In [7]:
# # test of polyglot detector

# arabic_text = u"""
# أفاد مصدر امني في قيادة عمليات صلاح الدين في العراق بأن " القوات الامنية تتوقف لليوم
# الثالث على التوالي عن التقدم الى داخل مدينة تكريت بسبب
# انتشار قناصي التنظيم الذي يطلق على نفسه اسم "الدولة الاسلامية" والعبوات الناسفة
# والمنازل المفخخة والانتحاريين، فضلا عن ان القوات الامنية تنتظر وصول تعزيزات اضافية ".
# """

# detector = Detector(arabic_text)
# print(detector.language)


#    TODO: figure out UTF-8 ERROR or just do try catch exception and return 
#    unknown or something
#    below code does not work


# df.track_name = df.track_name.astype(str)
# df['poly_obj'] = df.track_name.apply(lambda x: Detector(x, quiet=True))
# df['Track-lang'] = df['poly_obj'].apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
# df['Track-LangConfidence'] = df['poly_obj'].apply( lambda x: x.language.confidence)

'\n   TODO: figure out UTF-8 ERROR or just do try catch exception and return \n   unknown or something\n   below code does not work\n'

In [8]:
# helper function to one hot encode genre
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

def preprocess(df_input):

    time_sig_encoding = { '0/4' : 0, '1/4' : 1, 
                         '3/4' : 3, '4/4' : 4,
                         '5/4' : 5}

    key_encoding = { 'A' : 0, 'A#' : 1, 'B' : 2,
                    'C' : 3,  'C#' : 4,  'D' : 5,
                    'D#' : 6, 'E' : 7, 'F' : 8,
                    'F#' : 9, 'G' : 10, 'G#' : 11 }

    mode_encoding = { 'Major':0, 'Minor':1}      

    df['key'] = df['key'].map(key_encoding)
    df['time_signature'] = df['time_signature'].map(time_sig_encoding)
    df['mode'] = df['mode'].map(mode_encoding)
    df = encode_and_bind(df, 'genre')
    
df = preprocess(raw_df)

UnboundLocalError: local variable 'df' referenced before assignment

In [5]:
df.head(2)

AttributeError: 'NoneType' object has no attribute 'head'

In [14]:
df.sort_values('popularity', ascending=False)

AttributeError: 'NoneType' object has no attribute 'sort_values'

In [10]:
df.genre.value_counts()

AttributeError: 'NoneType' object has no attribute 'genre'

# MODELING: Nearest Neighbors
resources: https://scikit-learn.org/stable/modules/neighbors.html

In [ ]:
# instantiate model
neigh = NearestNeighbors(n_neighbors=11)

features = list(df.columns[4:])
X = df[features].values
# y = df[target]

X.shape # y.shape
# target = 'track_id'

%time
neigh.fit(X) # NN doesn't need to fit Y

# Model Test on Song Example

In [ ]:
# highest ranking popularity song from Ariana Grande
test_song = X[9027]
df.iloc[9027]

In [ ]:
distance, neighbors = neigh.kneighbors(np.array([test_song]))
distance

In [ ]:
neighbors

In [ ]:
song_list = []
for idx in neighbors[0][2:]: # this way excludes itself
    row = df.iloc[idx]
    print(f'Artist: {row.artist_name} - Track: {row.track_name}')
    song_list.append((row.artist_name, row.track_name))
song_list

# Helper Function to Return 10 Nearest Neighbors

In [ ]:
"""
function for what was done manually above
generalized for other dataframes instead of hard coded therefore:
takes in a dataframe, a feature set array, and a song ID
and returns a list of tuples of artist name and track names
"""
from typing import List, Tuple

def closest_ten(df: pd.DataFrame, X_array: np.ndarray ,song_id: int) -> List[Tuple] :
    song = df.iloc[song_id]
    X_song = X[song_id]
    _, neighbors = neigh.kneighbors(np.array([X_song]))
    song_list = []
    for idx in neighbors[0][2:]: 
        row = df.iloc[idx]
       # print(f'Artist: {row.artist_name} - Track: {row.track_name}')
        song_list.append((row.artist_name, row.track_name))
    return song_list

In [ ]:
# example of function call 
closest_ten(df, X, 9027)

# PCA -> NN

In [ ]:
# make copy to transform in case you want to do stuff with the other one later in notebook
df_copy = df[features].copy()

In [ ]:
from numpy import array, mean, std, cov
from numpy.linalg import eig
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
means = df_copy.mean()
# print("\n Means:", means)
scaler = StandardScaler()

In [ ]:
pca_X = scaler.fit_transform(df_copy.values)
pca = PCA()
df_pca = pca.fit(pca_X)

In [ ]:
print(pca.explained_variance_)

In [ ]:
%time
pca_neigh = NearestNeighbors(n_neighbors=11)
pca_neigh.fit(pca_X) # NN doesn't need to fit Y

In [ ]:
# refactor later if have time
# from sklearn.pipeline import make_pipeline
# pipeline = make_pipeline(
#     StandardScaler(),
#     NearestNeighbors())

# pipeline.fit(X)

# Auto Encoder -> NN

In [ ]:
from keras.models import Model

In [ ]:
encoding_dim = 10 # compression of factor 4.1 assuming input of 41

# Export Model with Joblib

In [ ]:
filename = 'NearestNeighborGenres.sav'

In [ ]:
joblib.dump(neigh, filename)

In [ ]:
pca_filename = 'PCANearestNeighborGenres.sav'
joblib.dump(pca_neigh, pca_filename)

In [ ]:
# one hot encoding, rolled back
# def encode_and_bind(original_dataframe, feature_to_encode):
#     dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
#     res = pd.concat([original_dataframe, dummies], axis=1)
#     return(res)

# df = encode_and_bind(df, 'key')
# df = encode_and_bind(df, 'mode')
# df = encode_and_bind(df, 'time_signature')

# Export Model with Pickle

In [ ]:
import pickle

model = neigh

filename = 'filename.pkl'

with open(filename, 'wb') as f:
    pickle.dump(model, f)
#then send this pkl file to DE team

#DE team:
#Make sure to save the pkl file in the same folder as the app

#To read it into the app:

# with open(filename, 'rb') as f:
#     model = pickle.load(f)